# Clean AFT Dataset
**Author:** Jim Maddock

**Last Updated:** 5-3-20

**Description:** Import the initial English AFT dataset (downloaded from https://old.datahub.io/dataset/wikipedia-article-feedback-corpus) and create features for exploratory analysis and classifier.  The processed dataset should include the following **feature fields**:
* unique ID
* original ID (these are non-unique but could be useful for debugging)
* article and revision ID
* comment (might have to remove stop tokens)
* article topic from https://github.com/wikimedia/drafttopic
* article features - TBD
* vandalism score from https://github.com/wikimedia/draftquality

The dataset should also include the following **class fields**:
* helpful/un-helpful
* non-actionable

In [47]:
import csv
import pandas as pd
import numpy as np
import uuid
import random
import requests
import time
from oresapi import Session
import concurrent.futures
import logging
import sys

import matplotlib.pyplot as plt
import matplotlib

matplotlib.style.use('ggplot')

In [48]:
LOGFILE = './logs/clean_dataset_5-7-20.log'

logger = logging.getLogger(__name__)
handler = logging.FileHandler(LOGFILE)

formatter = logging.Formatter(fmt='[%(levelname)s %(asctime)s] %(message)s',
                              datefmt='%m/%d/%Y %I:%M:%S %p')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [49]:
# import dataset from csv
FILEPATH = '/srv/aft/raw/dump_03-24-20.csv'

dtypes = {
    'aft_id':object
}

df = pd.read_csv(FILEPATH,escapechar='\\', encoding='latin-1', dtype=dtypes)

In [50]:
# remove all comments that do not have a helpful or unhelpful label
df = df.loc[(df['aft_helpful'] > 0) | (df['aft_unhelpful'] > 0)]

In [51]:
# generate a unique ID
# make sure to use a random seed so this is reproducable
rd = random.Random()
rd.seed(0)
df['UUID'] = df.apply(lambda x: uuid.UUID(int=rd.getrandbits(128)), axis=1)
df = df.reset_index()

In [52]:
# get vandalism score for each comment using draft quality
# PRE: comment text, not including stop words
# POST: return the positive and negative scores for the comment

# Ignore and log errors

BASE_URL = "https://ores.wikimedia.org/v3/scores/enwiki/57185234/draftquality"

def threadedVandalismScoreHandler(df, max_workers=4, handle_errors=False):
    
    to_process = df[['aft_comment','UUID']].iterrows()
    score_list = []
    processed = 0
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:

        future_to_url = {executor.submit(getVandalismScore, row[1]): row for row in to_process}
        for future in concurrent.futures.as_completed(future_to_url):
            try:
                score_dict = future.result()
                score_list.append(score_dict)
            except:
                if handle_errors:
                    pass
                else:
                    raise
            processed += 1
            
            #if processed % 100 == 0:
            #    print('scored {0} documents'.format(processed))
    
    score_df = pd.DataFrame(score_list)
    return score_df

# handle and log all errors
def getVandalismScore(row):
    try:
        score_json = vandalismScoreRequest(row['aft_comment'])
        score_dict = vandalismJsonToDict(score_json)
        score_dict['UUID'] = row['UUID']
        return score_dict
    except:
        logger.warning(row['UUID'])
        raise

def formatCommentText(comment_text):
    comment_text = str(comment_text)
    #comment_text = bytes(comment_text, 'latin-1').decode('utf-8')
    comment_text = comment_text.replace('\n','')
    comment_text = comment_text.replace('\t','')
    comment_text = comment_text.replace('"','')
    #comment_text = comment_text.replace("\'","")
    #comment_text = comment_text.replace('"','')
    #comment_text = comment_text.replace('”','')
    comment_text = '"{0}"'.format(comment_text)
    return comment_text
    

def vandalismScoreRequest(comment_text):
    comment_text = formatCommentText(comment_text)
    params = {
        'datasource.revision.text':comment_text
    }
    r = requests.get(BASE_URL,params=params)
    if not r:
        raise requests.RequestException()
    json_response = r.json()
    return json_response

def vandalismJsonToDict(json):
    base = json['enwiki']['scores']['57185234']['draftquality']['score']
    score_dict = {
        'dq_pred':base['prediction'],
        'dq_prob_OK':base['probability']['OK'],
        'dq_prob_attack':base['probability']['attack'],
        'dq_prob_spam':base['probability']['spam'],
        'dq_prob_vandalism':base['probability']['vandalism']
    }
    return score_dict

#threadedVandalismScoreHandler(test)

In [53]:
def getTopics(df):
    rev_id_list = df['aft_page_revision'].unique().tolist()
    
    session = Session("https://ores.wikimedia.org",user_agent="")
    results = session.score("enwiki", ["drafttopic"], rev_id_list)

    topic_list = []
    for rev_id, result in zip(rev_id_list, results):
        score_dict = {
            'aft_page_revision':rev_id
        }
        if 'error' in result['drafttopic']:
            score_dict['dt_pred'] = None
        elif len(result['drafttopic']['score']['prediction']) < 1:
            score_dict['dt_pred'] = None
        else:
            try:
                score_dict['dt_pred'] = result['drafttopic']['score']['prediction'][0],
            except:
                print(result)
                raise
        topic_list.append(score_dict)
    
    topic_df = pd.DataFrame(topic_list)
    return topic_df

#getTopics(test)

In [54]:
def addPredColumns(df):
    vandalism_pred_df = threadedVandalismScoreHandler(df, max_workers=8, handle_errors=True)
    topic_pred_df = getTopics(df)

    df = df.merge(vandalism_pred_df, on='UUID', how='outer')
    df = df.merge(topic_pred_df, on='aft_page_revision', how='outer')
    return df

In [55]:
FILE_OUTPUT_PATH = '/srv/aft/processed/chunks/chunk_{0}_5-7-20.csv'
CHUNK_SIZE = 100

def chunkAndProcess(df, fileout = None, chunk_size = 1000):
    df_list = [df[i:i+chunk_size] for i in range(0,df.shape[0],chunk_size)]
    for i, df_chunk in enumerate(df_list):
        try:
            result_chunk = addPredColumns(df_chunk)
            if fileout:
                result_chunk.to_csv(fileout.format(i), index = False)
            print('processed chunk {0}'.format(i))
        except:
            logger.warning('error in chunk {0}'.format(i))
            logger.warning(sys.exc_info())
            pass
    
#test = df[1000:2000]
#chunkAndProcess(test, fileout = FILE_OUTPUT_PATH, chunk_size=CHUNK_SIZE)

In [56]:
chunkAndProcess(df, fileout = FILE_OUTPUT_PATH, chunk_size=CHUNK_SIZE)

processed chunk 0
processed chunk 1
processed chunk 2
processed chunk 3
processed chunk 4
processed chunk 5
processed chunk 6
processed chunk 7
processed chunk 8
processed chunk 9
processed chunk 10


processed chunk 11


processed chunk 12
processed chunk 13
processed chunk 14
processed chunk 15
processed chunk 16
processed chunk 17


processed chunk 18
processed chunk 19


processed chunk 20


...

processed chunk 1143


processed chunk 1144


processed chunk 1145


processed chunk 1146


processed chunk 1147
processed chunk 1148
processed chunk 1149


In [57]:
len(df)

114984